### Importação de arquivo

In [ ]:
import sys
import CityHub
import networkx as nx
import numpy as np
import osmnx as ox
#import pickle
import shapely
#from shapely.geometry import Point
import geopandas as gpd
#from shapely import io
import pandas as pd
import umap

In [ ]:
%%time
cg=CityHub.CityHub('Maceió')
G=cg.city_street_graph

In [ ]:
type(G)

In [ ]:
len(G)

### Functions to verify whether laplacian is symetric and triangular

In [ ]:
# https://mail.python.org/pipermail/scipy-dev/2014-October/020120.html
from scipy.sparse import coo_matrix
def is_symmetric(m):
  if m.shape[0] != m.shape[1]:
    raise ValueError('m must be a square matrix')
  if not isinstance(m, coo_matrix):
    m = coo_matrix(m)
  r, c, v = m.row, m.col, m.data
  tril_no_diag = r > c
  triu_no_diag = c > r

  if triu_no_diag.sum() != tril_no_diag.sum():
    return False
  rl = r[tril_no_diag]
  cl = c[tril_no_diag]
  vl = v[tril_no_diag]
  ru = r[triu_no_diag]
  cu = c[triu_no_diag]
  vu = v[triu_no_diag]

  sortl = np.lexsort((cl, rl))
  sortu = np.lexsort((ru, cu))
  vl = vl[sortl]
  vu = vu[sortu]

  check = np.allclose(vl, vu)

  return check

def is_triangular(A):
  m = coo_matrix(A)
  r, c, v = m.row, m.col, m.data
  tril_no_diag = r > c
  triu_no_diag = c > r
  if triu_no_diag.sum()==0 or tril_no_diag.sum()==0 :
    return True
  return False

## Laplacian / Fiedler

In [ ]:
# The graph is connected?
print(nx.is_connected(G))
print(nx.number_connected_components(G))

In [ ]:
import osmnx as ox
import math

'''' If you want to use the average distance between all point pairs as sigma (slow)
soma=0.0
for u in G.nodes:
    for v in G.nodes:
        if u!=v:
            vertA = cg.city_vert_list[cg.city_vert_nxind_to_ind_dict[u]]
            vertB = cg.city_vert_list[cg.city_vert_nxind_to_ind_dict[v]]            
            soma+= ox.distance.great_circle_vec(vertA[0],vertA[1],vertB[0],vertB[1])
sigma=soma/(len(G.nodes)*(len(G.nodes)-1))
'''

sigma=5000

# Inverse length
for u in G.edges:
   G[u[0]][u[1]][0]["inv_length"] = 1/G[u[0]][u[1]][0]["length"] 
# Gaussian:
#    G[u[0]][u[1]][0]["inv_length"] = math.exp(-(G[u[0]][u[1]][0]["length"]**2)/(sigma**2)) 
    
laplacian = nx.laplacian_matrix(G,weight='inv_length')

# Is Symmetric?
print(is_symmetric(laplacian))
# Is Triangular?
print(is_triangular(laplacian))

In [ ]:
from scipy.sparse.linalg import eigs
eig_values, eig_vectors = eigs(laplacian.astype(float),k=2, which='LM', sigma=0)
fiedler_pos = np.where(eig_values.real == np.sort(eig_values.real)[1])[0][0]
fiedler_vector = np.transpose(eig_vectors)[fiedler_pos]

print("Fiedler vector: " + str(fiedler_vector.real))
fiedler_vector_real = fiedler_vector.real
fiedler_sorted_index_list = np.argsort(np.argsort(fiedler_vector_real))

i=0
for u in G.nodes:
    G.nodes[u]['fiedler_index']=fiedler_sorted_index_list[i]
    i=i+1
    
inverse_fiedler_sorted_index_list = [0] * len(cg.city_vert_list)
for i in range(len(fiedler_sorted_index_list)):
    inverse_fiedler_sorted_index_list[fiedler_sorted_index_list[i]]=i

# Sorting the nodes according to several techniques

## Random

In [ ]:
random_index_list = np.random.permutation(len(cg.city_vert_list))

## t-SNE 1D

In [ ]:
from sklearn.preprocessing import StandardScaler
import openTSNE

nodes2d = np.array(cg.city_vert_list)

gpdnodes = gpd.GeoDataFrame(geometry = gpd.points_from_xy(nodes2d[:,0], nodes2d[:,1]), crs = "EPSG:4326")
gpdnodes = gpdnodes.to_crs("EPSG:20824") # reprojetando para ficar melhor para Buenos Aires
nodesproj = gpdnodes.geometry.apply(lambda g: pd.Series(g.coords[0]))
scaler = StandardScaler()
scaler.fit(nodesproj)
nodesprojsca = scaler.transform(nodesproj)

'''
print("Calculando affinity")
#njobs = 32
aff3k = openTSNE.affinity.PerplexityBasedNN(
    nodesprojsca,
    perplexity=3000,
    n_jobs=32,
    random_state=0,
)
'''
print("Calculando TSNE")
#n_jobs=32
X_embedding = openTSNE.TSNE(
    n_components = 1,
    perplexity = 127,
    n_jobs=10,
    verbose=True,
).fit(nodesprojsca)


# Exibir ordenacao no mapa

print("Gerando dataframe")

nodesdf = pd.DataFrame(nodes2d)
nodesdf['y'] = nodesproj[0]
nodesdf['x'] = nodesproj[1]
nodesdf["tsne"] = X_embedding
nodesdf = nodesdf.sort_values('tsne').reset_index()
nodesdf["new_index"] = nodesdf.index

In [ ]:
import folium
import branca
import branca.colormap as cm

mi= 0
ma= nodesdf.shape[0] - 1

colormap = cm.LinearColormap(colors=['darkblue', 'blue', 'cyan', 'yellow', 'orange', 'red'],
                             index=[mi, ma/5, ma/5*2, ma/5*3, ma/5*4, ma], vmin=mi, vmax=ma,
                             caption='TSNE perplexity 500 on Buenos Aires')

Lats = nodes2d[:,0]
Longs = nodes2d[:,1]
med_lat = 0.5*(np.max(Lats)+np.min(Lats))
med_lon = 0.5*(np.max(Longs)+np.min(Longs))

m = folium.Map(location=[med_lat, med_lon], zoom_start=12, tiles='CartoDB positron')

for index, row in nodesdf.iterrows():
    marker = folium.CircleMarker([row[0], row[1]], radius=1, color=colormap(index))
    popup = folium.Popup(str(row['index']))
    marker.add_child(popup)
    m.add_child(marker)
   
m.add_child(colormap)
m.save('tsne_map_maceio2.html')
m

In [ ]:
df = nodesdf.sort_values(by=['index'])
tsne_sorted_index_list = np.array(df['new_index'])
inverse_tsne_sorted_index_list = [0] * len(cg.city_vert_list)
for i in range(len(tsne_sorted_index_list)):
    inverse_tsne_sorted_index_list[tsne_sorted_index_list[i]]=i

## UMAP

In [ ]:
umap_model = umap.UMAP(n_components=1, n_neighbors = 750, min_dist = 0.5, random_state=42)
umap_embedding = umap_model.fit_transform(nodesprojsca)

In [ ]:
nodesdf_umap = pd.DataFrame(nodes2d)
nodesdf_umap['y'] = nodesproj[0]
nodesdf_umap['x'] = nodesproj[1]
nodesdf_umap["umap"] = umap_embedding
nodesdf_umap = nodesdf_umap.sort_values('umap').reset_index()
nodesdf_umap["new_index"] = nodesdf_umap.index
nodesdf_umap

In [ ]:
import folium
import branca
import branca.colormap as cm

mi= 0
ma= nodesdf.shape[0] - 1

colormap = cm.LinearColormap(colors=['darkblue', 'blue', 'cyan', 'yellow', 'orange', 'red'],
                             index=[mi, ma/5, ma/5*2, ma/5*3, ma/5*4, ma], vmin=mi, vmax=ma,
                             caption='TSNE perplexity 5000')

Lats = nodes2d[:,0]
Longs = nodes2d[:,1]
med_lat = 0.5*(np.max(Lats)+np.min(Lats))
med_lon = 0.5*(np.max(Longs)+np.min(Longs))

m = folium.Map(location=[med_lat, med_lon], zoom_start=12, tiles='CartoDB positron')

for index, row in nodesdf_umap.iterrows():
    marker = folium.CircleMarker([row[0], row[1]], radius=1, color=colormap(index))
    popup = folium.Popup(str(index))
    marker.add_child(popup)
    m.add_child(marker)
   
m.add_child(colormap)
m.save('umap_map_maceio2.html')
m

In [ ]:
df = nodesdf_umap.sort_values(by=['index'])
umap_sorted_index_list = np.array(df['new_index'])
inverse_umap_sorted_index_list = [0] * len(cg.city_vert_list)
for i in range(len(umap_sorted_index_list)):
    inverse_umap_sorted_index_list[umap_sorted_index_list[i]]=i

## Metric for sorting quality: embedding -> latlong (FORWARD METRIC)

### original

In [ ]:
window_size = int(len(cg.city_vert_list)/100)

In [ ]:
print(window_size)
print(int(window_size/2))

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances,haversine_distances
from sklearn.neighbors import NearestNeighbors

In [ ]:
df

In [ ]:
originaldf = df[[0,1,'x','y']].copy()
originaldf.index = df['index'].values
originaldf

In [ ]:
cg.city_vert_list[1]

In [ ]:
def calculate_diagonal(list_of_indices):
    Lats = [cg.city_vert_list[j][0] for j in list_of_indices]
    Longs = [cg.city_vert_list[j][1] for j in list_of_indices]
    min_rad = [math.radians(_) for _ in [min(Lats),min(Longs)]]
    max_rad = [math.radians(_) for _ in [max(Lats),max(Longs)]]
    
    return haversine_distances([min_rad,max_rad])[0,1]* 6371000/1000

In [ ]:
def calculate_diagonal_using_projection(list_of_indices):
    min_x = min(originaldf.loc[list_of_indices]['x'])
    max_x = max(originaldf.loc[list_of_indices]['x'])
    min_y = min(originaldf.loc[list_of_indices]['y'])
    max_y = max(originaldf.loc[list_of_indices]['y'])

    return np.sqrt((max_x - min_x)**2 + (max_y - min_y)**2)

In [ ]:
def distance_on_earth(row1, row2):
    rad1 = [math.radians(row1[0]), math.radians(row1[1])]
    rad2 = [math.radians(row2[0]), math.radians(row2[1])]
    return haversine_distances([rad1,rad2])[0,1]* 6371000/1000

In [ ]:
distance_on_earth(df[[0,1]].iloc[0], df[[0,1]].iloc[1])

In [ ]:
%%time
knn = NearestNeighbors(n_neighbors=window_size, algorithm='auto')
knn.fit(originaldf[['x','y']])
neighbors = knn.kneighbors(originaldf[['x','y']], return_distance = False)
best_diagonal = np.apply_along_axis(calculate_diagonal, 1, neighbors)

In [ ]:
diagonals_original = []

for i in range(0,len(cg.city_vert_list)-window_size,1):
    Lats = [cg.city_vert_list[j][0] for j in range(i,i+window_size)]
    Longs = [cg.city_vert_list[j][1] for j in range(i,i+window_size)]
    min_rad = [math.radians(_) for _ in [min(Lats),min(Longs)]]
    max_rad = [math.radians(_) for _ in [max(Lats),max(Longs)]]
    
    diag = haversine_distances([min_rad,max_rad])[0,1]* 6371000/1000
    norm_diag = diag / best_diagonal[i + int(window_size/2)]
    
    diagonals_original.append(norm_diag)

### Fiedler

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances,haversine_distances
import math


diagonals_fiedler = []

for i in range(0,len(cg.city_vert_list)-window_size,1):
    Lats = [cg.city_vert_list[j][0] for j in inverse_fiedler_sorted_index_list[i:i+window_size]]
    Longs = [cg.city_vert_list[j][1] for j in inverse_fiedler_sorted_index_list[i:i+window_size]]
    min_rad = [math.radians(_) for _ in [min(Lats),min(Longs)]]
    max_rad = [math.radians(_) for _ in [max(Lats),max(Longs)]]
    
    diag = haversine_distances([min_rad,max_rad])[0,1]* 6371000/1000
    norm_diag = diag / best_diagonal[inverse_fiedler_sorted_index_list[i + int(window_size/2)]]
    
    diagonals_fiedler.append(norm_diag)

### t-SNE

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances,haversine_distances
import math


diagonals_tsne = []

for i in range(0,len(cg.city_vert_list)-window_size,1):
    Lats = [cg.city_vert_list[j][0] for j in inverse_tsne_sorted_index_list[i:i+window_size]]
    Longs = [cg.city_vert_list[j][1] for j in inverse_tsne_sorted_index_list[i:i+window_size]]
    min_rad = [math.radians(_) for _ in [min(Lats),min(Longs)]]
    max_rad = [math.radians(_) for _ in [max(Lats),max(Longs)]]
    
    diag = haversine_distances([min_rad,max_rad])[0,1]* 6371000/1000
    norm_diag = diag / best_diagonal[inverse_tsne_sorted_index_list[i + int(window_size/2)]]
    
    diagonals_tsne.append(norm_diag)

### UMAP

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances,haversine_distances
import math


diagonals_umap = []

for i in range(0,len(cg.city_vert_list)-window_size,1):
    Lats = [cg.city_vert_list[j][0] for j in inverse_umap_sorted_index_list[i:i+window_size]]
    Longs = [cg.city_vert_list[j][1] for j in inverse_umap_sorted_index_list[i:i+window_size]]
    min_rad = [math.radians(_) for _ in [min(Lats),min(Longs)]]
    max_rad = [math.radians(_) for _ in [max(Lats),max(Longs)]]
    
    diag = haversine_distances([min_rad,max_rad])[0,1]* 6371000/1000
    norm_diag = diag / best_diagonal[inverse_umap_sorted_index_list[i + int(window_size/2)]]
    
    diagonals_umap.append(norm_diag)

### Random

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances,haversine_distances

diagonals_random = []

for i in range(0,len(cg.city_vert_list)-window_size,1):
#for i in range(0,10000,1000):
    Lats = [cg.city_vert_list[j][0] for j in random_index_list[i:i+window_size]]
    Longs = [cg.city_vert_list[j][1] for j in random_index_list[i:i+window_size]]
    min_rad = [math.radians(_) for _ in [np.min(Lats),np.min(Longs)]]
    max_rad = [math.radians(_) for _ in [np.max(Lats),np.max(Longs)]]
    
    diag = haversine_distances([min_rad,max_rad])[0,1]* 6371000/1000
    norm_diag = diag / best_diagonal[random_index_list[i + int(window_size/2)]]
    
    diagonals_random.append(norm_diag)

In [ ]:
import statistics
print('Fiedler - média:',statistics.mean(diagonals_fiedler))
print('Fiedler - mediana:',statistics.median(diagonals_fiedler))
print('t-SNE - média:',statistics.mean(diagonals_tsne))
print('t-SNE - mediana:',statistics.median(diagonals_tsne))
print('UMAP - média:',statistics.mean(diagonals_umap))
print('UMAP - mediana:',statistics.median(diagonals_umap))
print('Original - média:',statistics.mean(diagonals_original))
print('Original - mediana:',statistics.median(diagonals_original))
print('Random - média:',statistics.mean(diagonals_random))
print('Random - mediana:',statistics.median(diagonals_random))

In [ ]:
print(min(diagonals_tsne))
print(max(diagonals_tsne))
print(len(diagonals_tsne))
print(len(cg.city_vert_list)-window_size)

In [ ]:
pd.Series(diagonals_tsne).plot.box()

In [ ]:
data = pd.Series(diagonals_tsne)
Q1   = data.quantile(0.25)
Q3   = data.quantile(0.75)
IQR  = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR
print(lower_bound, upper_bound)

In [ ]:
nodesdf

In [ ]:
import folium
import branca
import branca.colormap as cm

data = pd.Series(diagonals_tsne)
Q1   = data.quantile(0.25)
Q3   = data.quantile(0.75)
IQR  = Q3 - Q1
mi = Q1 - 1.5 * IQR
ma = Q3 + 1.5 * IQR

colormap = cm.LinearColormap(colors=['#fffdd0', 'red'],
                             index=[mi, ma], vmin=mi, vmax=ma,
                             caption='TSNE forward measure')

Lats = nodes2d[:,0]
Longs = nodes2d[:,1]
med_lat = 0.5*(np.max(Lats)+np.min(Lats))
med_lon = 0.5*(np.max(Longs)+np.min(Longs))

m = folium.Map(location=[med_lat, med_lon], zoom_start=12, tiles='CartoDB positron')

for esquina in range(len(cg.city_vert_list) - window_size):
    window_middle_i = inverse_tsne_sorted_index_list[esquina+int(window_size/2)]
    marker = folium.CircleMarker(cg.city_vert_list[window_middle_i], radius=1, color=colormap(diagonals_tsne[esquina]),opacity=0.5+((diagonals_tsne[esquina] - mi)*0.5/(ma-mi))).add_to(m)  
    popup = folium.Popup(str(window_middle_i))
    marker.add_child(popup)
    m.add_child(marker)
    
m.add_child(colormap)
m.save('tsne_map_maceio2_forward.html')
m

In [ ]:
nodesdf

In [ ]:
Lats = [cg.city_vert_list[j][0] for j in range(len(cg.city_vert_list))]
Longs = [cg.city_vert_list[j][1] for j in range(len(cg.city_vert_list))]
med_lat = 0.5*(np.max(Lats)+np.min(Lats))
med_lon = 0.5*(np.max(Longs)+np.min(Longs))

m = folium.Map(location=[med_lat+0.08, med_lon-0.05], zoom_start=12, tiles='CartoDB positron')

i_middle_original = 13274

folium.CircleMarker((cg.city_vert_list[i_middle_original][0],cg.city_vert_list[i_middle_original][1]), radius=6, color='green',opacity=1).add_to(m)  

i_middle = np.where(np.array(inverse_tsne_sorted_index_list) == i_middle_original)[0][0]
i = i_middle - int(window_size/2)

Lats = [cg.city_vert_list[j][0] for j in inverse_tsne_sorted_index_list[i:i+window_size]]
Longs = [cg.city_vert_list[j][1] for j in inverse_tsne_sorted_index_list[i:i+window_size]]

min_rad = [math.radians(_) for _ in [np.min(Lats),np.min(Longs)]]
max_rad = [math.radians(_) for _ in [np.max(Lats),np.max(Longs)]]
print(haversine_distances([min_rad,max_rad])[0,1]* 6371000/1000)

for k in range(window_size):
    folium.CircleMarker((Lats[k],Longs[k]), radius=4, color='red',opacity=1).add_to(m)  

    
neighbors = knn.kneighbors(originaldf.loc[[i_middle_original]][['x','y']], return_distance = False)[0]
Lats = [cg.city_vert_list[j][0] for j in neighbors]
Longs = [cg.city_vert_list[j][1] for j in neighbors]

for k in range(window_size):
    folium.CircleMarker((Lats[k],Longs[k]), radius=2, color='blue',opacity=1).add_to(m)  


m.add_child(colormap)
m

In [ ]:
import folium
import branca
import branca.colormap as cm

data = pd.Series(diagonals_fiedler)
Q1   = data.quantile(0.25)
Q3   = data.quantile(0.75)
IQR  = Q3 - Q1
mi = 0 #Q1 - 1.5 * IQR
ma = 8 # Q3 + 1.5 * IQR

colormap = cm.LinearColormap(colors=['#fffdd0', 'red'],
                             index=[mi, ma], vmin=mi, vmax=ma,
                             caption='Fiedler forward measure')

Lats = nodes2d[:,0]
Longs = nodes2d[:,1]
med_lat = 0.5*(np.max(Lats)+np.min(Lats))
med_lon = 0.5*(np.max(Longs)+np.min(Longs))

m = folium.Map(location=[med_lat, med_lon], zoom_start=12, tiles='CartoDB positron')

for esquina in range(len(cg.city_vert_list) - window_size):
    window_middle_i = inverse_fiedler_sorted_index_list[esquina+int(window_size/2)]
    marker = folium.CircleMarker(cg.city_vert_list[window_middle_i], radius=1, color=colormap(diagonals_fiedler[esquina]),opacity=0.5+((diagonals_fiedler[esquina] - mi)*0.5/(ma-mi))).add_to(m)  
    popup = folium.Popup(str(window_middle_i))
    marker.add_child(popup)
    m.add_child(marker)
    
m.add_child(colormap)
m.save('fiedler_map_maceio2_forward.html')
m

In [ ]:
import folium
import branca
import branca.colormap as cm

data = pd.Series(diagonals_tsne)
Q1   = data.quantile(0.25)
Q3   = data.quantile(0.75)
IQR  = Q3 - Q1
mi = 0# Q1 - 1.5 * IQR
ma = 8# Q3 + 1.5 * IQR

colormap = cm.LinearColormap(colors=['#fffdd0', 'red'],
                             index=[mi, ma], vmin=mi, vmax=ma,
                             caption='UMAP forward measure')

Lats = nodes2d[:,0]
Longs = nodes2d[:,1]
med_lat = 0.5*(np.max(Lats)+np.min(Lats))
med_lon = 0.5*(np.max(Longs)+np.min(Longs))

m = folium.Map(location=[med_lat, med_lon], zoom_start=12, tiles='CartoDB positron')

for esquina in range(len(cg.city_vert_list) - window_size):
    window_middle_i = inverse_umap_sorted_index_list[esquina+int(window_size/2)]
    marker = folium.CircleMarker(cg.city_vert_list[window_middle_i], radius=1, color=colormap(diagonals_umap[esquina]),opacity=0.5+((diagonals_umap[esquina] - mi)*0.5/(ma-mi))).add_to(m)  
    popup = folium.Popup(str(window_middle_i))
    marker.add_child(popup)
    m.add_child(marker)
    
m.add_child(colormap)
m.save('umap_map_maceio2_forward.html')
m

In [ ]:
import matplotlib.pyplot as plt

In [ ]:

plt.hist([diagonals_fiedler, diagonals_tsne,diagonals_umap, diagonals_original, diagonals_random], bins=40, label=['fiedler', 't-sne','umap','original', 'random'], color=['blue','orange','red', 'green','purple'],range=[0,30])
#plt.hist([diagonals_random], bins=40, label=['random'], color=['green'],range=[0,40])
plt.legend(loc='upper right')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(4, 3))
plt.boxplot([diagonals_fiedler, diagonals_tsne,diagonals_umap, diagonals_original, diagonals_random], labels =['fiedler', 't-sne','umap','original', 'random'], showfliers = False)
plt.xlabel("Index ordering")
plt.ylabel("D")
plt.savefig('BuAi_forward.pdf', dpi=300, format='pdf', bbox_inches='tight')
plt.show()

## Metric for quality: latlong -> embedding (INVERSE METRIC)

### Fiedler, original and random

In [ ]:
%%time
max_inverse_ind_dist_fiedler = []
max_inverse_ind_dist_original = []
max_inverse_ind_dist_random = []
max_inverse_ind_dist_tsne = []
max_inverse_ind_dist_umap = []

for i in range(0,len(cg.city_vert_list)):
    ps=nx.single_source_dijkstra_path(cg.city_street_graph,cg.city_vert_ind_to_nxind_dict[i],cutoff=0.5*1000.0,weight='length')
    result_nodes = []
    for k in ps.keys():
        result_nodes.append(cg.city_vert_nxind_to_ind_dict[k])  
    inds_sorted_fiedler = [fiedler_sorted_index_list[j] for j in result_nodes]
    inds_sorted_tsne = [tsne_sorted_index_list[j] for j in result_nodes]
    inds_sorted_umap = [umap_sorted_index_list[j] for j in result_nodes]
    inds_random = [random_index_list[j] for j in result_nodes]
    max_inverse_ind_dist_fiedler.append((np.max(inds_sorted_fiedler)-np.min(inds_sorted_fiedler))/len(inds_sorted_fiedler))  
    max_inverse_ind_dist_tsne.append((np.max(inds_sorted_tsne)-np.min(inds_sorted_tsne))/len(inds_sorted_tsne))  
    max_inverse_ind_dist_umap.append((np.max(inds_sorted_umap)-np.min(inds_sorted_umap))/len(inds_sorted_umap))  
    max_inverse_ind_dist_original.append((np.max(result_nodes)-np.min(result_nodes))/len(result_nodes))
    max_inverse_ind_dist_random.append((np.max(inds_random)-np.min(inds_random))/len(inds_random))  

In [ ]:
plt.hist([max_inverse_ind_dist_fiedler,max_inverse_ind_dist_tsne,max_inverse_ind_dist_umap,max_inverse_ind_dist_original,max_inverse_ind_dist_random], bins=40, label=['fiedler','t-sne','umap','original','random'], color=['blue', 'orange','red','green', 'purple'],range=[0,50])
plt.legend(loc='upper right')
plt.show()

In [ ]:
import statistics
print('Fiedler - média:',statistics.mean(max_inverse_ind_dist_fiedler))
print('Fiedler - mediana:',statistics.median(max_inverse_ind_dist_fiedler))
print('t-SNE - média:',statistics.mean(max_inverse_ind_dist_tsne))
print('t-SNE - mediana:',statistics.median(max_inverse_ind_dist_tsne))
print('UMAP - média:',statistics.mean(max_inverse_ind_dist_umap))
print('UMAP - mediana:',statistics.median(max_inverse_ind_dist_umap))
print('Original - média:',statistics.mean(max_inverse_ind_dist_original))
print('Original - mediana:',statistics.median(max_inverse_ind_dist_original))
print('Random - média:',statistics.mean(max_inverse_ind_dist_random))
print('Random - mediana:',statistics.median(max_inverse_ind_dist_random))

In [ ]:
plt.figure(figsize=(10, 6))
plt.boxplot([max_inverse_ind_dist_fiedler,max_inverse_ind_dist_tsne,max_inverse_ind_dist_umap,max_inverse_ind_dist_original,max_inverse_ind_dist_random], labels=['fiedler','t-sne','umap','original','random'])
plt.show()

In [ ]:
plt.figure(figsize=(4,3))
plt.boxplot([max_inverse_ind_dist_fiedler,max_inverse_ind_dist_tsne,max_inverse_ind_dist_umap], labels=['fiedler','t-sne','umap'], showfliers = False)
plt.xlabel("Index ordering")
plt.ylabel('$D^{inv}$')
plt.savefig('BuAi_inverse.pdf', dpi=300, format='pdf', bbox_inches='tight')
plt.show()

## Fiedler ordering map

In [ ]:
import folium
import branca
import branca.colormap as cm

#colormap = cm.LinearColormap(colors=['blue','yellow'],vmin=np.amin(sorted_index_list) ,vmax=np.amax(sorted_index_list))
mi= np.amin(fiedler_sorted_index_list)
ma = np.amax(fiedler_sorted_index_list)
colormap = cm.LinearColormap(colors=['darkblue', 'blue', 'cyan', 'yellow', 'orange', 'red'],
                             index=[mi, ma/5, ma/5*2, ma/5*3, ma/5*4, ma], vmin=mi, vmax=ma,
                             caption='')

Lats = [cg.city_vert_list[j][0] for j in range(len(cg.city_vert_list))]
Longs = [cg.city_vert_list[j][1] for j in range(len(cg.city_vert_list))]
med_lat = 0.5*(np.max(Lats)+np.min(Lats))
med_lon = 0.5*(np.max(Longs)+np.min(Longs))

m = folium.Map(location=[med_lat, med_lon], zoom_start=12, tiles='CartoDB positron')

for esquina in range(len(cg.city_vert_list)):
    marker = folium.CircleMarker(cg.city_vert_list[inverse_fiedler_sorted_index_list[esquina]], radius=1, color=colormap(esquina),opacity=1)
    popup = folium.Popup(str(esquina))
    marker.add_child(popup)
    m.add_child(marker) 
   
m.add_child(colormap)
m.save('fiedler_map_maceio2.html')
m

## t-SNE ordering map

In [ ]:
import folium
import branca
import branca.colormap as cm

#colormap = cm.LinearColormap(colors=['blue','yellow'],vmin=np.amin(sorted_index_list) ,vmax=np.amax(sorted_index_list))
min= np.amin(tsne_sorted_index_list)
max = np.amax(tsne_sorted_index_list)
colormap = cm.LinearColormap(colors=['darkblue', 'blue', 'cyan', 'yellow', 'orange', 'red'],
                             index=[min, max/5, max/5*2, max/5*3, max/5*4, max], vmin=min, vmax=max,
                             caption='Fiedler index')

Lats = [cg.city_vert_list[j][0] for j in range(len(cg.city_vert_list))]
Longs = [cg.city_vert_list[j][1] for j in range(len(cg.city_vert_list))]
med_lat = 0.5*(np.max(Lats)+np.min(Lats))
med_lon = 0.5*(np.max(Longs)+np.min(Longs))

m = folium.Map(location=[med_lat, med_lon], zoom_start=12, tiles='CartoDB positron')

for esquina in range(len(cg.city_vert_list)):
     folium.CircleMarker(cg.city_vert_list[inverse_tsne_sorted_index_list[esquina]], radius=1, color=colormap(esquina),opacity=1).add_to(m)  

   
m.add_child(colormap)
m
m.save('fiedler_map_tsne.html')

## tsne inverse metric map

In [ ]:
import folium
import branca
import branca.colormap as cm

#colormap = cm.LinearColormap(colors=['blue','yellow'],vmin=np.amin(sorted_index_list) ,vmax=np.amax(sorted_index_list))
mn = np.amin(0)
mx = 50#np.max(max_inverse_ind_dist_fiedler)/16
colormap = cm.LinearColormap(colors=['#fffdd0', 'red'],
                             index=[mn, mx], vmin=mn, vmax=mx,
                             caption='inverse indices distance through tsne error')

Lats = [cg.city_vert_list[j][0] for j in range(len(cg.city_vert_list))]
Longs = [cg.city_vert_list[j][1] for j in range(len(cg.city_vert_list))]
med_lat = 0.5*(np.max(Lats)+np.min(Lats))
med_lon = 0.5*(np.max(Longs)+np.min(Longs))

m = folium.Map(location=[med_lat, med_lon], zoom_start=12, tiles='CartoDB positron')

for esquina in range(len(cg.city_vert_list)):
    folium.CircleMarker(cg.city_vert_list[esquina], radius=1, color=colormap(max_inverse_ind_dist_tsne[esquina]), opacity = 0.5+((max_inverse_ind_dist_tsne[esquina] - 0)*0.5/(50-0))).add_to(m)  
   
m.add_child(colormap)
m.save('tsne_maceio2_inverse_measure_map.html')
m

In [ ]:
import folium
import branca
import branca.colormap as cm

#colormap = cm.LinearColormap(colors=['blue','yellow'],vmin=np.amin(sorted_index_list) ,vmax=np.amax(sorted_index_list))
mn = np.amin(0)
mx = 50#np.max(max_inverse_ind_dist_fiedler)/16
colormap = cm.LinearColormap(colors=['#fffdd0', 'red'],
                             index=[mn, mx], vmin=mn, vmax=mx,
                             caption='inverse indices distance through fiedler error')

Lats = [cg.city_vert_list[j][0] for j in range(len(cg.city_vert_list))]
Longs = [cg.city_vert_list[j][1] for j in range(len(cg.city_vert_list))]
med_lat = 0.5*(np.max(Lats)+np.min(Lats))
med_lon = 0.5*(np.max(Longs)+np.min(Longs))

m = folium.Map(location=[med_lat, med_lon], zoom_start=12, tiles='CartoDB positron')

for esquina in range(len(cg.city_vert_list)):
    folium.CircleMarker(cg.city_vert_list[esquina], radius=1, color=colormap(max_inverse_ind_dist_fiedler[esquina]), opacity = 0.5+((max_inverse_ind_dist_fiedler[esquina] - 0)*0.5/(50-0))).add_to(m)  
   
m.add_child(colormap)
m.save('fiedler_maceio2_inverse_measure_map.html')
m

In [ ]:
import folium
import branca
import branca.colormap as cm

#colormap = cm.LinearColormap(colors=['blue','yellow'],vmin=np.amin(sorted_index_list) ,vmax=np.amax(sorted_index_list))
mn = np.amin(0)
mx = 50#np.max(max_inverse_ind_dist_fiedler)/16
colormap = cm.LinearColormap(colors=['#fffdd0', 'red'],
                             index=[mn, mx], vmin=mn, vmax=mx,
                             caption='inverse indices distance through umap error')

Lats = [cg.city_vert_list[j][0] for j in range(len(cg.city_vert_list))]
Longs = [cg.city_vert_list[j][1] for j in range(len(cg.city_vert_list))]
med_lat = 0.5*(np.max(Lats)+np.min(Lats))
med_lon = 0.5*(np.max(Longs)+np.min(Longs))

m = folium.Map(location=[med_lat, med_lon], zoom_start=12, tiles='CartoDB positron')

for esquina in range(len(cg.city_vert_list)):
    folium.CircleMarker(cg.city_vert_list[esquina], radius=1, color=colormap(max_inverse_ind_dist_umap[esquina]), opacity = 0.5+((max_inverse_ind_dist_umap[esquina] - 0)*0.5/(50-0))).add_to(m)  
   
m.add_child(colormap)
m.save('umap_maceio2_inverse_measure_map.html')
m